In [126]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF, WideDeep, DIN  # pure data, 
from libreco.evaluation import evaluate


import tensorflow as tf

In [127]:
data = pd.read_parquet("../data/dicoding_user_item_rating.gzip")
data.rename(columns = 
    {
        "user_id":"user",
        "course_id":"item",
        "rating":"label"
    }, inplace= True
) 

In [128]:
data

,user,item,graduated_at,label
0,623699,14,2020-03-10 11:45:50,5
1,406371,14,2020-03-10 08:44:09,4
2,1946,14,2020-03-11 13:24:10,5
3,186713,14,2020-03-11 17:36:04,4
4,462580,14,2020-03-11 08:43:36,4
...,...,...,...,...
303951,3244648,615,2023-09-04 17:09:09,5
303952,3258243,615,2023-09-04 19:12:11,5
303953,3270793,615,2023-09-04 20:11:31,5
303954,2547435,615,2023-09-04 20:38:22,5


In [129]:
train_data, eval_data, test_data = random_split(data[[
    "user", "item", "label"
]], multi_ratios=[0.8, 0.1, 0.1])

In [130]:
# LibRecommender Datatype


train_data, data_info= DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)

In [131]:
### NCF


ncf = NCF(
    task="rating",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=16,
    n_epochs=10,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
    
)

In [132]:
ncf.fit(
    train_data,
    neg_sampling=False, #for rating, this param is false else True
    verbose=0,
    eval_data=eval_data,
    metrics=["loss"],
    
)

# do final evaluation on test data
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["loss"],

)


Training start time: 2023-10-11 12:58:17


ValueError: Variable embedding/user_embeds_var already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/abilfad/miniconda3/envs/recommender-system/lib/python3.10/site-packages/libreco/layers/embedding.py", line 17, in embedding_lookup
    embed_var = tf.get_variable(
  File "/home/abilfad/miniconda3/envs/recommender-system/lib/python3.10/site-packages/libreco/algorithms/ncf.py", line 121, in build_model
    user_embeds = embedding_lookup(
  File "/home/abilfad/miniconda3/envs/recommender-system/lib/python3.10/site-packages/libreco/bases/tf_base.py", line 128, in fit
    self.build_model()
  File "/tmp/ipykernel_5170/3040933838.py", line 3, in <module>
    ncf.fit(
  File "/home/abilfad/miniconda3/envs/recommender-system/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


# light gcn